In [1]:
# Usual imports
import secml
import numpy as np
from tqdm import tqdm
from scipy.special import softmax
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from joblib import Parallel, delayed
import pickle
import os
import pandas as pd
import csv

# SecML
from secml.ml.features.normalization import CNormalizerMinMax
from secml.ml.peval.metrics import CMetricAccuracy
from secml.array import CArray
from secml.ml.classifiers import CClassifierPyTorch

# RobustBench
import robustbench
from robustbench.utils import load_model
from secml.utils import fm
from secml import settings

# Albi utils
from utils_attacks import *
from utils_CP import *

2025-04-19 10:27:43.640980: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-19 10:27:43.856791: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-19 10:27:43.867069: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.10/dist-packages/torch/lib:/usr/local/lib/python3.10/dist-pa

2025-04-19 10:27:55,188 - py.warnings - WARNING - /usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



In [2]:
from secml.data.loader.c_dataloader_cifar import CDataLoaderCIFAR100
lr,_ = CDataLoaderCIFAR100().load()

n_tr = 1000  # Number of training set samples
n_val = 50  # Number of validation set samples
n_ts = 1000  # Number of test set samples
n_cl = 1000 # Number of calibration set samples

n = n_tr + n_val + n_cl + n_ts

# Shuffle before splitting
random_state = 999
rng = np.random.default_rng(seed=random_state)
shuffled_indices = rng.permutation(lr.X.shape[0]).tolist()
lr = lr[shuffled_indices, :]

# Split the dataset
tr = lr[:n_tr, :]
vl = lr[n_tr:n_tr + n_val, :]
cl = lr[n_tr + n_val:n_tr + n_val + n_cl, :]
ts = lr[n_tr + n_val + n_cl:n, :]

# Normalize the features in `[0, 1]`
tr.X /= 255
vl.X /= 255
ts.X /= 255
cl.X /= 255


## Load a Model

### Wang: WideResNet-70-16
### RA (linf): 42.66%

In [6]:
output_dir = fm.join(settings.SECML_MODELS_DIR, 'robustbench')
model_wang = load_model(model_name='Wang2023Better_WRN-70-16', dataset='cifar100', threat_model='Linf', model_dir=output_dir)
clf_wang = CClassifierPyTorch(model_wang, input_shape=(3,32,32), pretrained=True, softmax_outputs = True)

#### Modas


In [20]:
from robustbench.model_zoo.enums import ThreatModel
from robustbench.utils import load_model, clean_accuracy
from robustbench.model_zoo import model_dicts
from robustbench.model_zoo.enums import BenchmarkDataset

print(model_dicts[BenchmarkDataset.cifar_100].keys())  
print(model_dicts[BenchmarkDataset.cifar_100][ThreatModel.corruptions].keys())

odict_keys([<ThreatModel.Linf: 'Linf'>, <ThreatModel.corruptions: 'corruptions'>])
odict_keys(['Diffenderfer2021Winning_LRR', 'Diffenderfer2021Winning_LRR_CARD_Deck', 'Diffenderfer2021Winning_Binary', 'Diffenderfer2021Winning_Binary_CARD_Deck', 'Gowal2020Uncovering_Linf', 'Gowal2020Uncovering_extra_Linf', 'Hendrycks2020AugMix_WRN', 'Hendrycks2020AugMix_ResNeXt', 'Addepalli2021Towards_PARN18', 'Addepalli2021Towards_WRN34', 'Modas2021PRIMEResNet18', 'Addepalli2022Efficient_WRN_34_10'])


In [29]:
output_dir = fm.join(settings.SECML_MODELS_DIR, 'robustbench')
model_modas = load_model(model_name='Addepalli2022Efficient_WRN_34_10', dataset='cifar100', threat_model='corruptions', model_dir=output_dir)
clf_modas = CClassifierPyTorch(model_modas, input_shape=(3,32,32), pretrained=True, softmax_outputs = True)

from secml.ml.peval.metrics import CMetricAccuracy
metric = CMetricAccuracy()

# Compute predictions on a test set
y_pred = clf_modas.predict(ts.X)

# Evaluate the accuracy of the classifier
acc = metric.performance_score(y_true=ts.Y, y_pred=y_pred)

print("Accuracy on test set: {:.2%}".format(acc))

Downloading...
From (original): https://drive.google.com/uc?id=1-3c-iniqNfiwGoGPHC3nSostnG6J9fDt
From (redirected): https://drive.google.com/uc?id=1-3c-iniqNfiwGoGPHC3nSostnG6J9fDt&confirm=t&uuid=2fda8470-7505-4345-8b38-4db919e93e89
To: /home/acarlevaro/secml-data/models/robustbench/cifar100/corruptions/Addepalli2022Efficient_WRN_34_10.pt

  0%|          | 0.00/185M [00:00<?, ?B/s]
  1%|▏         | 2.62M/185M [00:00<00:07, 24.9MB/s]
  3%|▎         | 5.24M/185M [00:00<00:13, 13.0MB/s]
  4%|▍         | 7.34M/185M [00:00<00:12, 14.4MB/s]
  7%|▋         | 13.1M/185M [00:00<00:08, 20.6MB/s]
  9%|▊         | 15.7M/185M [00:00<00:08, 20.2MB/s]
 12%|█▏        | 21.5M/185M [00:01<00:07, 22.7MB/s]
 14%|█▍        | 25.7M/185M [00:01<00:06, 24.9MB/s]
 17%|█▋        | 30.9M/185M [00:01<00:04, 30.9MB/s]
 19%|█▊        | 34.6M/185M [00:01<00:05, 27.3MB/s]
 20%|██        | 37.7M/185M [00:01<00:05, 26.0MB/s]
 23%|██▎       | 42.5M/185M [00:01<00:05, 25.4MB/s]
 25%|██▍       | 45.6M/185M [00:01<00:05, 2

Accuracy on test set: 98.30%


In [12]:
import shutil

def evaluate_attacks(model_name, norm_name, attack_configs, cl, ts, clf, alpha=0.1, base_output_dir="./Results"):
    """
    Evaluate different adversarial attacks on a given model and dataset.
    """
    n_cl = cl.X.shape[0]
    
    # Prepare output directory
    save_path = os.path.join(base_output_dir, model_name, norm_name)
    if os.path.exists(save_path):
        shutil.rmtree(save_path)
    os.makedirs(save_path, exist_ok=True)
    csv_file = os.path.join(save_path, "results_all.csv")
    
    # Run attacks
    cl_att_dict = attack_dataset(cl, clf, attack_configs, desc="Running attacks", n_jobs=1)
    ts_att_dict = attack_dataset(ts, clf, attack_configs, desc="Running attacks", n_jobs=1)
    
    # Process results
    results = []
    
    if isinstance(cl_att_dict, dict):
        attack_types = cl_att_dict.keys()
    else:
        attack_types = [get_single_attack_key(attack_configs[0])]
        cl_att_dict = {attack_types[0]: cl_att_dict}
        ts_att_dict = {attack_types[0]: ts_att_dict}
    
    for attack_type in attack_types:
        cl_att = cl_att_dict[attack_type]
        ts_att = ts_att_dict[attack_type]
        
        cl_att_scores = compute_score(cl, cl_att, clf)
        cl_scores = compute_score(cl, cl, clf)
        
        # Compute quantiles
        q_level = np.ceil((n_cl + 1) * (1 - alpha)) / n_cl
        qhat = np.quantile(cl_scores, q_level, method='higher')
        qhat_A = np.quantile(cl_att_scores, q_level, method='higher')
        
        # Compute conformal sets
        att_conformal_sets,_ = compute_CP(ts_att, qhat_A, clf)
        cs_conformal_sets,_ = compute_CP(ts_att, qhat, clf)
        
        # Compute coverage and variance
        att_coverage = compute_covergae(ts, att_conformal_sets)
        att_coverage_var = compute_covergae_std(ts, att_conformal_sets)
        cs_coverage = compute_covergae(ts, cs_conformal_sets)
        cs_coverage_var = compute_covergae_std(ts, cs_conformal_sets)
        
        # Compute mean and variance of set sizes
        att_size_mean = mean_conformal_sets(att_conformal_sets)
        att_size_var = std_conformal_sets(att_conformal_sets) / 10
        cs_size_mean = mean_conformal_sets(cs_conformal_sets)
        cs_size_var = std_conformal_sets(cs_conformal_sets) / 10
        
        results.append({
            "attack_type": "Vanilla",
            "coverage": f"{cs_coverage:.4f} ± {cs_coverage_var:.4f}",
            "size": f"{cs_size_mean:.4f} ± {cs_size_var:.4f}"
        })
        
        results.append({
            "attack_type": attack_type,
            "coverage": f"{att_coverage:.4f} ± {att_coverage_var:.4f}",
            "size": f"{att_size_mean:.4f} ± {att_size_var:.4f}"
        })
    
    # Save results to CSV
    file_exists = os.path.isfile(csv_file)
    with open(csv_file, mode="a", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=["attack_type", "coverage", "size"])
        if not file_exists:
            writer.writeheader()
        writer.writerows(results)
    
    print(f"Results saved to {csv_file}")


## Run them all

### Wang

In [ ]:
model_name = "Wang"

attack_configs = [ 
    {"attack_type": "PGD", "epsilon": 0.5, "step_size": 0.5/10, "steps": 10, "distance": "l2", "lb": 0.0, "ub": 1.0},
    {"attack_type": "FGM", "epsilon": 0.5, "distance": "l2", "lb": 0.0, "ub": 1.0},
    {"attack_type": "DeepFool", "epsilon": 0.5, "distance": "l2"},
    {"attack_type": "BasicIterative", "epsilon": 0.5, "distance": "l2"},
    {"attack_type": "CW"}
    #{"attack_type": "DDN", "epsilon": 0.5, "init_epsilon":0.01, "gamma":0.01, "steps":50, "lb":0.0, "ub":1.0},
    #{"attack_type": "EAD", "epsilon": 0.5, "binary_search_steps":15, "initial_stepsize":0.01, "confidence":0.0, "initial_const":0.01, "regularization":0.1, "steps":10, "lb":0.0, "ub":1.0 }
]

evaluate_attacks(model_name, "L2", attack_configs, cl, ts, clf_wang, alpha=0.1, base_output_dir="./Results")

eps_inf = 0.5/np.sqrt(32)

attack_configs_linf = [ 
    {"attack_type": "PGD", "epsilon": eps_inf, "step_size": eps_inf/10, "steps": 10, "distance": "linf", "lb": 0.0, "ub": 1.0},
    {"attack_type": "FGM", "epsilon": eps_inf, "distance": "linf", "lb": 0.0, "ub": 1.0},
    {"attack_type": "DeepFool", "epsilon": eps_inf, "distance": "linf"},
    {"attack_type": "BasicIterative", "epsilon": eps_inf, "distance": "linf"},

]

evaluate_attacks(model_name, "Linf", attack_configs_linf, cl, ts, clf_wang, alpha=0.1, base_output_dir="./Results")

eps_1 = 0.5*np.sqrt(32)

attack_configs_l1 = [ 
    {"attack_type": "PGD", "epsilon": eps_1, "step_size": eps_1/10, "steps": 10, "distance": "l1", "lb": 0.0, "ub": 1.0},
    {"attack_type": "FGM", "epsilon": eps_1, "distance": "l1", "lb": 0.0, "ub": 1.0},
    {"attack_type": "BasicIterative", "epsilon": eps_1, "distance": "l1"},

]

evaluate_attacks(model_name, "L1", attack_configs, cl, ts, clf_wang, alpha=0.1, base_output_dir="./Results")

Running attacks:   1%|          | 25/2500 [41:03<68:18:37, 99.36s/sample] 

#### Rice

In [ ]:
model_name = "Addepali"

attack_configs = [ 
    {"attack_type": "PGD", "epsilon": 0.5, "step_size": 0.5/10, "steps": 10, "distance": "l2", "lb": 0.0, "ub": 1.0},
    {"attack_type": "FGM", "epsilon": 0.5, "distance": "l2", "lb": 0.0, "ub": 1.0},
    {"attack_type": "DeepFool", "epsilon": 0.5, "distance": "l2", "steps": 10},
    {"attack_type": "BasicIterative", "epsilon": 0.5, "distance": "l2", "steps": 10},
    {"attack_type": "CW","step_size": 0.5/10, "steps": 10}
    #{"attack_type": "DDN", "epsilon": 0.5, "init_epsilon":0.01, "gamma":0.01, "steps":50, "lb":0.0, "ub":1.0},
    #{"attack_type": "EAD", "epsilon": 0.5, "binary_search_steps":15, "initial_stepsize":0.01, "confidence":0.0, "initial_const":0.01, "regularization":0.1, "steps":10, "lb":0.0, "ub":1.0 }
]


evaluate_attacks(model_name, "L2", attack_configs, cl, ts, clf_modas, alpha=0.1, base_output_dir="./Results")




Running attacks:  10%|█         | 105/1000 [20:09<3:13:23, 12.96s/sample]